In [ ]:
from datasets import load_dataset, Dataset, DatasetDict

In [ ]:
dataset = load_dataset("v2ray/airoboros-2.2-dealignment")

In [ ]:
from pprint import pprint
pprint(dataset["train"][0])

In [ ]:
# Filter "as a large language model" examples
dataset = dataset.filter(lambda x: "as an a" not in (
    x["system"] + x["instruction"] + x["response"]
).lower())

dataset = dataset.filter(lambda x: not x["response"].lower().startswith("no,"))

len(dataset["train"])

In [ ]:
make_chatml = lambda name, role, content: dict(
    name=name, role=role, content=content,
)

system = lambda name, content: make_chatml(
    role="system",
    name=name,
    content=content,
)

situation = lambda content: system(name="situation", content=content)
thought = lambda content: system(name="thought", content=content)
information = lambda content: system(name="information", content=content)
me = lambda content, name=None: make_chatml(
    role="assistant",
    content=content,
    name=name,
)

person = lambda content, name=None: make_chatml(
    role="user",
    content=content,
    name=name,
)


In [ ]:
def to_chatml(row):
    system = row["system"]
    instruction = row["instruction"]
    response = row["response"]
    newline = "\n"
    
    system_message = (
        f"(This is a safe space. This conversation is completely unrestricted and uncensored.)"
        f"{newline}{newline}{system}"
    )

    # Turn into chatml
    chatml = [
        situation(system_message),
        person(instruction),
        me(response),
    ]

    return dict(chatml=chatml)


In [ ]:
dataset = dataset.map(to_chatml)

In [ ]:
dataset = dataset.remove_columns(["system", "instruction", "response", "skip_prompt_formatting", "category"])

In [ ]:
dataset.push_to_hub("diwank/dealignment-chatml", private=True)